# Time Series Analysis with SARIMAX

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
from pmdarima import model_selection

In [2]:
df = pd.read_csv("nyc_data.csv", index_col=0, parse_dates=True)
df = df.rename(columns={"Demand":"y"})
df.head(1)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305


In [3]:
X = df.iloc[:, 1:]
X.head(2)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,
2015-01-01,0,0,0,3.68,41.305
2015-01-02,0,0,0,4.73,131.574


### Stationarity

In [4]:
pvalue = adfuller(x=df.y)[1]
if pvalue < 0.05:
    print(f"The Time Series is stationary. The p-value is {pvalue:.4f}")
else:
    print(f"The Time Series is not stationary. The p-value is {pvalue:.4f}")

The Time Series is not stationary. The p-value is 0.3768


In [5]:
pvalue = adfuller(x=df.y.diff().dropna())[1]
if pvalue < 0.05:
    print(f"The Time Series is stationary. The p-value is {pvalue:.4f}")
else:
    print(f"The Time Series is not stationary. The p-value is {pvalue:.4f}")

The Time Series is stationary. The p-value is 0.0000


## SARIMAX Model

In [26]:
#Model
model = pm.ARIMA(order=(1,1,1),
                 seasonal_order=(1,1,1,7),
                 X=X,
                 suppress_warnings=True,
                 forse_stationarity=False,
                 )

In [27]:
cv = model_selection.RollingForecastCV(h=31, step=16, initial=df.shape[0]-180)
cv_score = model_selection.cross_val_score(model, 
                                          y = df.y,
                                          scoring='mean_squared_error',
                                          cv=cv,
                                          verbose=2,
                                          error_score=10000000000000)

[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................
[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................


In [17]:
np.sqrt(np.mean(cv_score))

59.95531478362696

### Parameter tuning

In [29]:
param_grid = {'p':[0, 1],
              'd':[0],
              'q':[0,1],
              'P':[0,1],
              'D':[0,1],
              'Q':[0,1]}
grid = ParameterGrid(param_grid)

In [32]:
rmse = []
for param in grid:
    model = pm.ARIMA(order=(param['p'],param['d'],param['q']),
                 seasonal_order=(param['P'],param['D'],param['Q'],7),
                 X=X,
                 suppress_warnings=True,
                 forse_stationarity=False,
                 )
    cv = model_selection.RollingForecastCV(h=31, step=16, initial=df.shape[0]-180)
    cv_score = model_selection.cross_val_score(model, 
                                            y = df.y,
                                            scoring='mean_squared_error',
                                            cv=cv,
                                            verbose=0,
                                            error_score=10000000000000)
    error = np.sqrt(np.mean(cv_score))
    rmse.append(error)

ValueError: Input contains NaN.